# Step 4: Create models with SD

In [1]:
%load_ext jupyternotify
import pandas as pd 
import os 
import sys 
import pickle


from sklearn.metrics import (classification_report, 
                             roc_auc_score, 
                             matthews_corrcoef,
                             cohen_kappa_score)
sys.path.append('../src')
from utils import (getExperimentConfig, 
                   getPicklesFromDir, 
                   run_pycaret_setup, 
                   translate_model_name,
                   get_synthetic_filepaths_from_original_data_id,
                   convert_and_clean_dict)

from tuning_grids import Grids

from mlflow_manager import MLFlowManager

# Get global variables for the experiment
config = getExperimentConfig()
# Get folders
folders = config['folders']
# Load dataset specific settings (from the real-data)
dataset_settings = getPicklesFromDir(folders['settings_dir'])

<IPython.core.display.Javascript object>

In [ ]:
%%notify -m "Step 4: SD_models has finished!"

run_dataset = config['run_dataset']

for settings in dataset_settings:
        
    if run_dataset is not None and settings['meta']['id'] not in run_dataset:
        continue
        
    # update system_log name
    settings['setup_param']['system_log'] = folders['log_dir']+"Step4_SD"
    # disable saving train-test split data (to save space)
    settings['setup_param']['log_data'] = False
    
    # Get experiment logging
    experiment_name = f"{settings['meta']['id']}-{settings['meta']['name']}"
    mlflow = MLFlowManager(experiment_name)
    
    best_model_from_original_data = mlflow.get_best_model_hyperparameters()

    original_data = pd.read_csv(f"{folders['real_dir']}{settings['meta']['filename']}")
    y_test = original_data[settings['meta']['target']]
    x_test = original_data.drop(columns=[settings['meta']['target']])

    logg_tags = {'Dataset Type': 'synthetic'}
    mlflow.start_run('Synthetic data models', tags=logg_tags)
    
    synthetic_datasets = get_synthetic_filepaths_from_original_data_id(settings['meta']['id'])

    for sd_filename in synthetic_datasets:
        
        sd_id = os.path.splitext(sd_filename)[0]
        sd_path = folders['sd_dir']+sd_filename
        
        s = run_pycaret_setup(sd_path, settings['setup_param'])
        USI = s.get_config('USI')
        
        # Add custom tags to the logg, defining dataset type, and Id
        logg_tags = {
            'Dataset Type': 'synthetic',
            'Dataset ID': sd_id,
            'USI': USI
        }
        run_name = f"{sd_id}-models"
        mlflow.start_run(run_name, tags=logg_tags, nested=True)

        # TODO: need to fix, currently isnt working
        """
        ########### Test the model with best performance from best original dataset ###########
        ml_model = best_model_from_original_data['model']
        hyperparameters = best_model_from_original_data['params']
        hyperparameters = convert_and_clean_dict(hyperparameters)
        model_name = f"Original_{sd_id}{translate_model_name(ml_model)}"
        logg_tags['model']=model_name
        
        mlflow.start_run(model_name, tags=logg_tags, nested=True)

        # create & tune model
        tmp_model = s.create_model(ml_model)
        model=tmp_model.set_params(**hyperparameters)
        tuned_model = s.create_model(model)
    
        # get validation results
        val_df = s.pull()
        val_score = {}
        val_score['val_Accuracy'] = val_df['Accuracy']['Mean']
        val_score['val_F1-score'] = val_df['F1']['Mean']


        pred_model = s.predict_model(estimator=tuned_model, data=x_test)
        y_pred = pred_model['prediction_label']

        metrics =  classification_report(y_true=y_test, y_pred=y_pred, output_dict=True, digits=4)
        test_score = pd.DataFrame.from_dict(metrics).transpose()
        
        test_metrics = {
            "Accuracy": metrics['accuracy'],
            "F1-score": metrics['macro avg']['f1-score'],
            "MCC": matthews_corrcoef(y_true=y_test, y_pred=y_pred),
            "Kappa": cohen_kappa_score(y1=y_test, y2=y_pred)
        }

        # If there is a prediction_score in the from predict_model (sometimes there isn't)
        if 'prediction_score' in pred_model.columns:
            y_pred_score = pred_model['prediction_score']
            # If multiclass classification, set argument multi_class='one-vs-one'
            if original_data[settings['meta']['target']].nunique() > 2:
                m_class = 'ovo'
            else:
                m_class = 'raise'
            test_metrics['AUC'] = roc_auc_score(y_true=y_test, y_score=y_pred_score, multi_class=m_class)


        # log parameters     
        mlflow.log_params(tuned_model.get_params())
        # log performance
        mlflow.log_metric_report(test_metrics)
        mlflow.log_metrics(val_score)
        mlflow.log_score_report_to_html(val_df, "Validation")
        mlflow.log_score_report_to_html(test_score, "Test_score")
        # log model
        mlflow.log_model(model=tuned_model)
        # end logging run for the model
        mlflow.end_run()
        ########### End test hyper-param ###########
        """
        
        
        # Start testing all models
        for ml_model in config['clf']['ml_models']:
            #start log run
            logg_tags['model']=ml_model
            model_name = f"{sd_id}-{translate_model_name(ml_model)}"
            mlflow.start_run(model_name, tags=logg_tags, nested=True)
            
            # create & tune model
            model = s.create_model(ml_model)
            tune_grid = Grids.get_tuning_grid(ml_model)
            
            # Is buggy, use default tuning by pycaret
            #tuned_model = s.tune_model(model, **config['clf']['tuning_param'], custom_grid=tune_grid)
            tuned_model = s.tune_model(model, **config['clf']['tuning_param'])            
            
            # get validation results
            val_df = s.pull()
            val_score = {}
            val_score['val_Accuracy'] = val_df['Accuracy']['Mean']
            val_score['val_F1-score'] = val_df['F1']['Mean']


            pred_model = s.predict_model(estimator=tuned_model, data=x_test)
            y_pred = pred_model['prediction_label']

            metrics =  classification_report(y_true=y_test, y_pred=y_pred, output_dict=True, digits=4)
            test_score = pd.DataFrame.from_dict(metrics).transpose()
            
            test_metrics = {
                "Accuracy": metrics['accuracy'],
                "F1-score": metrics['macro avg']['f1-score'],
                "MCC": matthews_corrcoef(y_true=y_test, y_pred=y_pred),
                "Kappa": cohen_kappa_score(y1=y_test, y2=y_pred)
            }

            # If there is a prediction_score in the from predict_model (sometimes there isn't)
            if 'prediction_score' in pred_model.columns:
                y_pred_score = pred_model['prediction_score']
                # If multiclass classification, set argument multi_class='one-vs-one'
                if original_data[settings['meta']['target']].nunique() > 2:
                    m_class = 'ovo'
                else:
                    m_class = 'raise'
                test_metrics['AUC'] = roc_auc_score(y_true=y_test, y_score=y_pred_score, multi_class=m_class)


            # log parameters     
            mlflow.log_params(tuned_model.get_params())
            # log performance
            mlflow.log_metric_report(test_metrics)
            mlflow.log_metrics(val_score)
            mlflow.log_score_report_to_html(val_df, "Validation")
            mlflow.log_score_report_to_html(test_score, "Test_score")
            # log model
            mlflow.log_model(model=tuned_model)
            # end logging run for the model
            mlflow.end_run()
            
        # end logging run for SD_id
        #TODO: save best model for each sd_id-models
        # Save model details on the model with best accurracy under the the 'Original data models' run
        #best_run = mlflow.get_best_run_by_metric(metric_name='Accuracy')

        #mlflow.log_params(best_run.data.params)
        #mlflow.log_metrics(best_run.data.metrics)
        #mlflow.log_tag('model run name', best_run.data.tags['mlflow.runName'])
        #mlflow.log_tag('model', best_run.data.tags['model'])
        #mlflow.log_tag('model run id', best_run.info.run_id)
    
        mlflow.end_run()

    # end logging for the synthetic datasets based on original id
    mlflow.end_run()          

,Description,Value
0,Session id,2010
1,Target,Outcome
2,Target type,Binary
3,Original data shape,"(768, 9)"
4,Transformed data shape,"(768, 9)"
5,Transformed train set shape,"(614, 9)"
6,Transformed test set shape,"(154, 9)"
7,Numeric features,8
8,Preprocess,True
9,Imputation type,simple


,Accuracy,AUC,Recall,Prec.,F1,Kappa,MCC
Fold,,,,,,,
0,0.6613,0.6572,0.9487,0.6607,0.7789,0.1445,0.2004
1,0.6129,0.5591,0.9231,0.6316,0.7500,0.0120,0.0178
2,0.6129,0.5909,0.9231,0.6316,0.7500,0.0120,0.0178
3,0.6613,0.7949,1.0000,0.6500,0.7879,0.1070,0.2377
4,0.6393,0.7232,1.0000,0.6393,0.7800,0.0000,0.0000
5,0.6230,0.5169,0.9744,0.6333,0.7677,-0.0324,-0.0970
6,0.6721,0.6358,1.0000,0.6610,0.7959,0.1134,0.2451
7,0.6230,0.6876,0.9487,0.6379,0.7629,-0.0072,-0.0129
8,0.6721,0.7284,1.0000,0.6610,0.7959,0.1134,0.2451


,Accuracy,AUC,Recall,Prec.,F1,Kappa,MCC
Fold,,,,,,,
0,0.6613,0.6572,0.9487,0.6607,0.7789,0.1445,0.2004
1,0.6129,0.5591,0.9231,0.6316,0.7500,0.0120,0.0178
2,0.6129,0.5909,0.9231,0.6316,0.7500,0.0120,0.0178
3,0.6613,0.7949,1.0000,0.6500,0.7879,0.1070,0.2377
4,0.6393,0.7232,1.0000,0.6393,0.7800,0.0000,0.0000
5,0.6230,0.5169,0.9744,0.6333,0.7677,-0.0324,-0.0970
6,0.6721,0.6358,1.0000,0.6610,0.7959,0.1134,0.2451
7,0.6230,0.6876,0.9487,0.6379,0.7629,-0.0072,-0.0129
8,0.6721,0.7284,1.0000,0.6610,0.7959,0.1134,0.2451


Fitting 10 folds for each of 1 candidates, totalling 10 fits
Fitting 10 folds for each of 1 candidates, totalling 10 fits
Fitting 10 folds for each of 1 candidates, totalling 10 fits
Fitting 10 folds for each of 1 candidates, totalling 10 fits
Fitting 10 folds for each of 1 candidates, totalling 10 fits
Fitting 10 folds for each of 1 candidates, totalling 10 fits
Fitting 10 folds for each of 1 candidates, totalling 10 fits
Fitting 10 folds for each of 1 candidates, totalling 10 fits
Fitting 10 folds for each of 1 candidates, totalling 10 fits
Fitting 10 folds for each of 1 candidates, totalling 10 fits
Original model was better than the tuned model, hence it will be returned. NOTE: The display metrics are for the tuned model (not the original one).


,Accuracy,AUC,Recall,Prec.,F1,Kappa,MCC
Fold,,,,,,,
0,0.6774,0.6689,0.9231,0.6792,0.7826,0.2102,0.2522
1,0.5968,0.5552,0.9231,0.6207,0.7423,-0.0403,-0.0658
2,0.5968,0.5123,0.9487,0.6167,0.7475,-0.0631,-0.1402
3,0.6935,0.7475,0.9487,0.6852,0.7957,0.2420,0.3020
4,0.7049,0.6941,0.9744,0.6909,0.8085,0.2396,0.3251
5,0.5902,0.5390,0.8718,0.6296,0.7312,-0.0438,-0.0562
6,0.7377,0.6515,0.9231,0.7347,0.8182,0.3687,0.4012
7,0.6557,0.6935,0.8974,0.6731,0.7692,0.1431,0.1689
8,0.6230,0.6638,0.8718,0.6538,0.7473,0.0615,0.0726


...

,Accuracy,AUC,Recall,Prec.,F1,Kappa,MCC
Fold,,,,,,,
0,0.6774,0.6667,1.0000,0.6552,0.7917,0.1969,0.3304
1,0.6290,0.6502,1.0000,0.6230,0.7677,0.0506,0.1611
2,0.6290,0.5685,0.9474,0.6316,0.7579,0.0847,0.1295
3,0.6452,0.6886,1.0000,0.6333,0.7755,0.1003,0.2297
4,0.6557,0.6762,1.0000,0.6379,0.7789,0.1477,0.2824
5,0.5738,0.6002,0.8919,0.6000,0.7174,-0.0285,-0.0406
6,0.6066,0.6363,0.9459,0.6140,0.7447,0.0343,0.0578
7,0.6230,0.7190,0.9459,0.6250,0.7527,0.0824,0.1263
8,0.6230,0.6492,0.9459,0.6250,0.7527,0.0824,0.1263


,Accuracy,AUC,Recall,Prec.,F1,Kappa,MCC
Fold,,,,,,,
0,0.6774,0.6667,1.0000,0.6552,0.7917,0.1969,0.3304
1,0.6290,0.6502,1.0000,0.6230,0.7677,0.0506,0.1611
2,0.6290,0.5685,0.9474,0.6316,0.7579,0.0847,0.1295
3,0.6452,0.6886,1.0000,0.6333,0.7755,0.1003,0.2297
4,0.6557,0.6762,1.0000,0.6379,0.7789,0.1477,0.2824
5,0.5738,0.6002,0.8919,0.6000,0.7174,-0.0285,-0.0406
6,0.6066,0.6363,0.9459,0.6140,0.7447,0.0343,0.0578
7,0.6230,0.7190,0.9459,0.6250,0.7527,0.0824,0.1263
8,0.6230,0.6492,0.9459,0.6250,0.7527,0.0824,0.1263


Fitting 10 folds for each of 1 candidates, totalling 10 fits
Fitting 10 folds for each of 1 candidates, totalling 10 fits
Fitting 10 folds for each of 1 candidates, totalling 10 fits
Fitting 10 folds for each of 1 candidates, totalling 10 fits
Fitting 10 folds for each of 1 candidates, totalling 10 fits
Fitting 10 folds for each of 1 candidates, totalling 10 fits
Fitting 10 folds for each of 1 candidates, totalling 10 fits
Fitting 10 folds for each of 1 candidates, totalling 10 fits
Fitting 10 folds for each of 1 candidates, totalling 10 fits
Fitting 10 folds for each of 1 candidates, totalling 10 fits
Original model was better than the tuned model, hence it will be returned. NOTE: The display metrics are for the tuned model (not the original one).


,,
,,
Initiated,. . . . . . . . . . . . . . . . . .,14:01:29
Status,. . . . . . . . . . . . . . . . . .,Fitting 10 Folds
Estimator,. . . . . . . . . . . . . . . . . .,K Neighbors Classifier


Processing:   0%|          | 0/4 [00:00<?, ?it/s]